In [1]:
import numpy as n
import math
import glob
import cv2 as cv
import matplotlib.pyplot as plt
from PIL import Image

### Membentuk Matriks Co-occurence

Membentuk matriks co-occurence 0, 45, 90, dan 135 derajat berdasarkan gray-level dari citra.

In [2]:
def co_occurence(citra, rentang, derajat):
    matrixCO = n.zeros([rentang+1,rentang+1])
    p,l = citra.shape
    if(derajat == 0):
        for i in range (rentang+1):
            for j in range (rentang+1):
                jml = 0
                for x in range (p):
                    for y in range (l-1):
                        if(citra[x][y]==i and citra[x][y+1]==j):
                            jml += 1
                        if(citra[x][y]==j and citra[x][y+1]==i):
                            jml += 1
                matrixCO[i][j] = jml
    
    elif (derajat == 45):
        for i in range (rentang+1):
            for j in range (rentang+1):
                jml = 0
                for x in range (p-1):
                    for y in range (1,l):
                        if(citra[x][y]==i and citra[x+1][y-1]==j):
                            jml += 1
                        if(citra[x][y]==j and citra[x+1][y-1]==i):
                            jml += 1
                matrixCO[i][j] = jml
                
    elif (derajat == 90):
        for i in range (rentang+1):
            for j in range (rentang+1):
                jml = 0
                for x in range (p-1):
                    for y in range (l):
                        if(citra[x][y]==i and citra[x+1][y]==j):
                            jml += 1
                        if(citra[x][y]==j and citra[x+1][y]==i):
                            jml += 1
                matrixCO[i][j] = jml
    
    elif (derajat == 135):
        for i in range (rentang+1):
            for j in range (rentang+1):
                jml = 0
                for x in range (p-1):
                    for y in range (l-1):
                        if(citra[x][y]==i and citra[x+1][y+1]==j):
                            jml += 1
                        if(citra[x][y]==j and citra[x+1][y+1]==i):
                            jml += 1
                matrixCO[i][j] = jml
                
    return matrixCO

### Membentuk matriks GLCM

Melakukan transpose matriks co-occurence dan menjumlahkan hasilnya dengan matriks co-occurence awal.

In [3]:
def transpose(array):
    p,l = array.shape
    trans = n.zeros([p,l])
    for i in range (p):
        for j in range (l):
            trans[i][j] = array[j][i]
    return trans
def sumUp(a,b):
    p,l = a.shape
    hasilJumlah = n.zeros([p,l])
    for i in range (p):
        for j in range (l):
            hasilJumlah[i][j] = a[i][j] + b[i][j]
    return hasilJumlah

### Normalisasi

Mendapatkan matriks GLCM yang sudah dinormalisasi.

In [4]:
def normalisasi(array):
    p,l = array.shape
    hasilJumlah = 0
    for i in range (p):
        for j in range (l):
            hasilJumlah += array[i][j]
#     print (hasilJumlah)
    for i in range (p):
        for j in range (l):
            array[i][j] = array[i][j] / hasilJumlah
    return array
def getGLCM(citra, derajat):
    co = co_occurence(citra,n.amax(citra),derajat)
    transpose_co = transpose(co)
    simetris = sumUp(co,transpose_co)
    normal = normalisasi(simetris)
    return normal

### Ekstraksi Fitur GLCM

Fungsi-fungsi perhitungan fitur-fitur GLCM yaitu kontras, homogenitas, entropi, dan energi citra.

In [5]:
def hitungKontras(array):
    p,l = array.shape
    kontras = n.zeros([p,l])
    for i in range (p):
        for j in range (l):
            kontras[i][j] = array[i][j] * (pow((i-j),2))
    contrast = 0
    for i in range (p):
        for j in range (l):
            contrast += kontras[i][j]
    return contrast
def hitungHomogenitas(array):
    p,l = array.shape
    hg = n.zeros([p,l])
    for i in range (p):
        for j in range (l):
            hg[i][j] = array[i][j] / (1+((i-j)*(i-j)))
    homogen = 0
    for i in range (p):
        for j in range (l):
            homogen += hg[i][j]
    return homogen
def hitungEntropi(array):
    p,l = array.shape
    ent = n.zeros([p,l])
    for i in range (p):
        for j in range (l):
            if(array[i][j]!=0):
                ent[i][j] = -(array[i][j] * (math.log(array[i][j])))
#     print(ent)
    entropy = 0
    for i in range (p):
        for j in range (l):
            entropy += ent[i][j]
    return entropy
def hitungEnergi(array):
    p,l = array.shape
    eng = n.zeros([p,l])
    for i in range (p):
        for j in range (l):
            eng[i][j] = pow(array[i][j],2)
#     print(eng)
    energy = 0
    for i in range (p):
        for j in range (l):
            energy += eng[i][j]
    return energy

### Mendapatkan keseluruhan fitur

In [38]:
def ekstraksiFitur(matriks):
    baris = n.zeros([4])
    baris[0] = hitungKontras(matriks)
    baris[1] = hitungHomogenitas(matriks)
    baris[2] = hitungEntropi(matriks)
    baris[3] = hitungEnergi(matriks)
    return baris
import csv
def insertoCsv(data):
    row = data;
    with open('GLCM_Covid.csv', 'a', newline='') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(row)
#   print("succes")
        csvFile.close()

### Mendapatkan semua fitur dari masing-masing gambar

In [47]:
def ekstraksiData(data, label):
    iterasi = 0
    for image0 in data:
        glcm0 = getGLCM(image0,0)
        glcm45 = getGLCM(image0,45)
        glcm90 = getGLCM(image0,90)
        glcm135 = getGLCM(image0,135)  
        fitur0 = ekstraksiFitur(glcm0)
        fitur45 = ekstraksiFitur(glcm45)
        fitur90 = ekstraksiFitur(glcm90)
        fitur135 = ekstraksiFitur(glcm135)
        fitur = [label, fitur0[0], fitur0[1], fitur0[2], fitur0[3], fitur45[0], fitur45[1], fitur45[2], fitur45[3], fitur90[0],fitur90[1],fitur90[2],fitur90[3], fitur135[0],fitur135[1],fitur135[2],fitur135[3]]
#         print(fitur)
        insertoCsv(fitur)
#         iterasi+=1

## Load Data dan mengekstrak masing-masing fitur GLCM-nya

In [52]:
data = []
for filename in glob.glob('Covid*.png'):
#     imageObject = Image.open(filename).convert('L')
#     newsize = (128,128)
#     a = imageObject.resize(newsize)
    imgg = cv.imread(filename)
    a = cv.resize (imgg, (128,128))
    img = cv.cvtColor(a,cv.COLOR_BGR2GRAY)
    image = n.array(img)
    p,l = image.shape
    for i in range(p):
        for j in range (l):
            image[i][j] /= 16
    data.append(image)
ekstraksiData(data,1)

In [53]:
data = []
for filename in glob.glob('NonCovid*.png'):
#     imageObject = Image.open(filename).convert('L')
#     newsize = (128,128)
#     a = imageObject.resize(newsize)
    imgg = cv.imread(filename)
    a = cv.resize (imgg, (128,128))
    img = cv.cvtColor(a,cv.COLOR_BGR2GRAY)
    image = n.array(img)
    p,l = image.shape
    for i in range(p):
        for j in range (l):
            image[i][j] /= 16
    data.append(image)
ekstraksiData(data,0)